In [ ]:
import os
import sys
import csv
import requests
import json
import operator
import time
import datetime
import platform
import itertools

In [105]:
def writeText(text, path, mode = 'w'):     # function to work with text files
    with open (path, mode) as textout:
        textout.write((text))
        
def writeJson(json, path, mode = 'w'):          # function to work with Json files
    with open(path, mode) as file:
        file.write(json.dumps(json))
        
def writeCsv(listOut, outputFile):          # function to work with csv files
    import csv
    with open (outputFile, "w", newline='', encoding = 'utf-8') as outputfile:
        writer = csv.writer(outputfile, delimiter = ",")
        for element in listOut:
            writer.writerow(element)

In [106]:
def getTxt(path):                 # open txt file in read mode
    return open(path, 'r').read()

def getCsv(path, delim = ','):    # insert data of csv in lists 
    list_return = []
    with open (path, encoding = 'windows-1252') as file:
        csvreader = csv.reader(file, delimiter = delim)        
        for i, line in enumerate(csvreader):
            list_inner = []
            list_inner.append(line[0].replace('\"','').replace('\'','').replace('[','').replace(']','').strip().split(','))
            list_inner.append(line[1].replace('\"','').replace('\'','').replace('[','').replace(']','').strip().split(','))
            list_inner.append(line[2].replace('\"','').replace('\'','').replace('[','').replace(']','').strip().split(','))
            list_return.append(list_inner)
    
    
    return list_return

In [107]:
if platform.system() == 'Windows':           # platform specific paths for opening prep_out.csv
    prep_data = getCsv('..\\output\\prep_out.csv')
else:
    prep_data = getCsv('../output/prep_out.csv')

In [108]:
print(len(prep_data))     # get amount of lists (data sets)
print(prep_data[1141:1145]) # print for test purposes the 1141st list

3573
[[['agricultural', ' raw', ' materials', ' imports', ' merchandise', ' imports', ' countries'], ['agricultural', ' raw', ' materials', ' comprise', ' sitc', ' section', ' crude', ' materials', ' fuels', ' excluding', ' divisions', ' crude', ' fertilizers', ' minerals', ' excluding', ' coal', ' petroleum', ' precious', ' stones', ' metalliferous', ' ores', ' scrap'], ['']], [['agricultural', ' raw', ' materials', ' imports', ' merchandise', ' imports', ' countries', ' grouped', ' income', ' levels'], ['agricultural', ' raw', ' materials', ' comprise', ' sitc', ' section', ' crude', ' materials', ' fuels', ' excluding', ' divisions', ' crude', ' fertilizers', ' minerals', ' excluding', ' coal', ' petroleum', ' precious', ' stones', ' metalliferous', ' ores', ' scrap'], ['']], [['agricultural', ' raw', ' materials', ' imports', ' merchandise', ' imports', ' world', ' regions'], ['agricultural', ' raw', ' materials', ' comprise', ' sitc', ' section', ' crude', ' materials', ' fuels', 

Mein Ansatz hier wäre gewesen die erste Liste (title) in der Geamtliste zu ersetzen durch ein Set (um duplicate zu vermeiden) von 
der ersten Liste  (title) (prep_data[0]) und 
der 2ten liste (desc.) (prep_data[1])
Aber entweder der Code ist vollkommen falsch oder ich hab mich mit den Listen vertan. 

In [109]:
# print(list(prep_data[1][0] + prep_data[1][1]))
# print(set(prep_data[3572][0] + prep_data[3572][1]))

In [110]:
# for i, elem in enumerate(prep_data):
#    prep_data[i][0] = list(set(prep_data[i][0] + prep_data[i][1]))

In [111]:
# print(prep_data[2])

## Word2Vec

In [112]:
def getVecList(list_in):
    failcnt = 0
    failelemcnt = 0
    str_log = 'Started at:\n' + datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S') + '\n'
    print('Started at:')
    print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'),'\n')
    # List of Lists as input needed
    adress = 'http://word2vec.ai.wu.ac.at/googlenews/model?word='
#     adress = 'http://word2vec.ai.wu.ac.at/twitter27_200/model?word='
    list_return = [[],[],[]]
    list_failed = []
    
    
    
    for i, elem in enumerate(list_in):
        inner_feat1, inner_feat2, inner_label = [], [], []
        str_1 = '-------------------------------\n' + 'Element ' + str(i)
        #print(str_1)
        str_log += str_1 + '\n'
    
        
# name ----------------------------------------
        for str_elem in elem[0]:
            word = str_elem.strip() 
            try:
                inner_feat1.append([float(x) for x in requests.get((adress+word)).text.replace(' ','').replace('[','').replace(']','').split(',')])
            except:
                try:
                    inner_feat1.append([float(x) for x in requests.get((adress+word.title())).text.replace(' ','').replace('[','').replace(']','').split(',')])
                except Exception as e:
                    str_2 = 'fail\n' + adress+(word) + '\n' + str(e) + '\n' + requests.get((adress+word)).text
                    #print(str_2)
                    str_log += str_2 + '\n'
#                     inner_feat1.append([])
                    list_failed.append(word)
                    failcnt += 1
# desc ----------------------------------------
        for str_elem in elem[1]:
            word = str_elem.strip() 
            try:
                inner_feat2.append([float(x) for x in requests.get((adress+word)).text.replace(' ','').replace('[','').replace(']','').split(',')])
            except:
                try:
                    inner_feat2.append([float(x) for x in requests.get((adress+word.title())).text.replace(' ','').replace('[','').replace(']','').split(',')])
                except Exception as e:
                    str_3 = 'fail\n' + adress+(word) + '\n' + str(e) + '\n' + requests.get((adress+word)).text
                    #print(str_3)
                    str_log += str_3 + '\n'
#                     inner_feat2.append([])
                    list_failed.append(word)
                    failcnt += 1
# label ----------------------------------------
        for str_elem in elem[2]:
            word = str_elem.strip() 
            try:
                inner_label.append([float(x) for x in requests.get((adress+word)).text.replace(' ','').replace('[','').replace(']','').split(',')])
            except:
                try:
                    inner_label.append([float(x) for x in requests.get((adress+word.title())).text.replace(' ','').replace('[','').replace(']','').split(',')])
                except Exception as e:
                    str_4 = 'fail\n' + adress+(word) + '\n' + str(e) + '\n' + requests.get((adress+word)).text
                    #print(str_4)
                    str_log += str_4 + '\n'
#                     inner_label.append([])
                    list_failed.append(word)
                    failcnt += 1
        
        
        if (inner_feat1 and inner_feat2 and inner_label):
            list_return[0].append(inner_feat1)
            list_return[1].append(inner_feat2)
            list_return[2].append(inner_label)
        else:
            str_log += 'fail\nEmpty List can not be appended!\n'
            failelemcnt += 1
            
        
        
    print('Failed words: ', failcnt)
    print('Failed Elements: ', failelemcnt)
    str_log += '\nFinished at:\n' + datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
    writeText(str_log, '../output/log.txt')    
    print('\nEnded at:')
    print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')) 
    return list_return, list_failed

In [ ]:
vect_list = getVecList(prep_data)

Started at:
2018-11-06 15:11:20 



In [ ]:
feature1 = vect_list[0][0]
feature2 = vect_list[0][1]
labels = vect_list[0][2]
missing = vect_list[1]

In [ ]:
len(feature1)

In [ ]:
if platform.system() == 'Windows':
    writeCsv(feature1, '..\\output\\feat1.csv')
    writeCsv(feature2, '..\\output\\feat2.csv')
    writeCsv(labels, '..\\output\\labels.csv')
else:
    writeCsv(feature1, '../output/feat1.csv')
    writeCsv(feature2, '../output/feat2.csv')
    writeCsv(labels, '../output/labels.csv')

In [ ]:
print(feature1[0])
print('-----------------------------------')
print(feature2[0])
print('-----------------------------------')
print(labels[0])

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>



















